In [9]:
!pip install pyspark

## Import Libraries for Data Cleaning

In [10]:
import warnings

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import os

In [11]:
warnings.filterwarnings('ignore')

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()
df = spark.read.csv("drive/MyDrive/NYC Home/Data/NYC Taxi Duration.csv", header=True)
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- trip_duration: string (nullable = true)



In [14]:
df = df.drop(df._c0)
df = df.drop(df.id)

In [15]:
df.show()

+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1|-73.98215484619139| 40.76793670654297|-73.96463012695312|40.765602111816406|                 N|          455|
|        1|2016-06-12 00:43:35|2016-06-12 00:54:38|              1|-73.98041534423827|40.738563537597656|-73.99948120117188|40.731151580810554|                 N|          663|
|        2|2016-01-19 11:35:24|2016-01-19 12:10:48|              1|-73.97902679443358|40.763938903808594|-74.005332

## Check for unacceptable values

In [16]:
# Check for NaN or null values

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+------------------+-------------+
|vendor_id|pickup_datetime|dropoff_datetime|passenger_count|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+------------------+-------------+
|        0|              0|               0|              0|               0|              0|                0|               0|                 0|            0|
+---------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+------------------+-------------+



In [17]:
# Count the number of entries where trip_duration is less than 0.

df.filter(df.trip_duration <= 0).count()

0

In [18]:
# Checking for unique values in `store_and_fwd_flag`

df.select('store_and_fwd_flag').distinct().collect()

[Row(store_and_fwd_flag='Y'), Row(store_and_fwd_flag='N')]

In [19]:
# Checking for unique values in `vendor_id`

df.select('vendor_id').distinct().collect()

[Row(vendor_id='1'), Row(vendor_id='2')]

In [20]:
df.show()

+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1|-73.98215484619139| 40.76793670654297|-73.96463012695312|40.765602111816406|                 N|          455|
|        1|2016-06-12 00:43:35|2016-06-12 00:54:38|              1|-73.98041534423827|40.738563537597656|-73.99948120117188|40.731151580810554|                 N|          663|
|        2|2016-01-19 11:35:24|2016-01-19 12:10:48|              1|-73.97902679443358|40.763938903808594|-74.005332

In [21]:
if os.path.exists("drive/MyDrive/NYC Home/Data/NYC Taxi Duration Cleaned") == False:
    df.write.option("header", "true").csv("./NYC Taxi Duration Cleaned")